# WebNLG 

* We use https://github.com/git-cloner/llama-lora-fine-tuning/tree/main to finetune LLaMA-7B using the chat format

1. Clone the Repository and remove fschat from requirements.txt

In [ ]:
!git clone https://github.com/git-cloner/llama-lora-fine-tuning.git

2. Clone the FastChat library

In [ ]:
!git clone https://github.com/lm-sys/FastChat.git

3. Adapt the system message of the vicuna_v1.1 conversation template (this is used for finetuning) in fastchat/conversation.py

4. Build the FastChat library

In [ ]:
!pip3 install -e .

5. Install all dependencies of the llama-lora-fine-tuning repository

In [ ]:
!pip3 install -r requirements.txt

6. Run the fine-tuning command
* You might need to adapt some parameters based on your hardware i.e. GPU
* model_name_or_path is the path to the LLaMA weights
* data_path is the path to the fine-tuning dataset
* output_dir is the directory in which the adapter will be placed

In [ ]:
!deepspeed fastchat/train/train_lora.py \
    --deepspeed ./deepspeed-config.json \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --model_name_or_path ~/Development/LLM/LLAMA/7B/Transformer/ \
    --data_path ~/Development/LLM/FastChat/data/webnlg_finetune_dataset_chat.json \
    --fp16 True \
    --output_dir ./output/lora-adapter/webnlg-adapter \
    --num_train_epochs 5 \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --gradient_accumulation_steps 1\
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 1200 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --model_max_length 512 \
    --gradient_checkpointing True

7. Merge the LoRA adapter into the LLaMA weights

In [ ]:
!python3 -m fastchat.model.apply_lora --base /home/ubuntu/Development/LLM/LLAMA/7B/Transformer --target /home/ubuntu/Development/LLM/Lora/llama-lora-fine-tuning/output/lora-model/webnlg-model --lora /home/ubuntu/Development/LLM/Lora/llama-lora-fine-tuning/output/lora-adapter/webnlg-adapter